Import libraries

In [ ]:
import pandas as pd
import numpy as np
import recordlinkage

Specify path directories

In [ ]:
# Specify root folder
root_folder = r'Development\facility_mapping_old'

# Combined Excel file
new_excel_file_path = root_folder + r'\Data\master_file_new.xlsx'

# Zone file
zone_mappings_file_path = root_folder + r'\Output\zone_mappings.xlsx'

# Woreda file
woreda_mappings_file_path = root_folder + r'\Output\woreda_mappings.xlsx'

# HP mapping path
hp_mappings_file_path = root_folder + r'\Output\hp_final_mappings.xlsx'

# HP mapping path
hc_mappings_file_path = root_folder + '\Output\hc_final_mappings.xlsx'

Read in data and rename columns where necessary

In [ ]:
df_new_eCHIS_combined = pd.read_excel(new_excel_file_path, sheet_name='echis_master')
df_new_eCHIS_combined = df_new_eCHIS_combined.add_suffix('_echis')

df_dhis2 = pd.read_excel(new_excel_file_path, sheet_name='dhis2_master')
df_dhis2.rename(columns={'Region':'region_name_dhis2', 'Zone':'zone_name_dhis2', 'Woreda':'woreda_name_dhis2', 'PHCU':'hc_name_dhis2', 'Facility Name':'facility_name_dhis2'}, inplace=True)

print('# of records in DHIS2:\t\t', len(df_dhis2))

print('# of records in mfr:\t\t', len(df_new_eCHIS_combined))


Pre-Processing

In [ ]:
# Make adjustments for Dire Dawa
df_new_eCHIS_combined['zone_name_echis'] = df_new_eCHIS_combined[['woreda_name_echis','zone_name_echis','region_name_echis']].apply(lambda x : x.woreda_name_echis if x.region_name_echis == 'Dire Dawa' else x.zone_name_echis, axis=1)

print('# of potential health posts in mfr', len(df_new_eCHIS_combined))

Using recordlinkage to link dhis2 and echis regions

In [ ]:
region_e = df_new_eCHIS_combined[['region_name_echis']].drop_duplicates().dropna().reset_index(drop=True)
region_d = df_dhis2[['region_name_dhis2']].drop_duplicates().dropna().reset_index(drop=True)

print(region_e)
print(region_d)

indexer = recordlinkage.Index()
indexer.full()
region_candidate_links = indexer.index(region_e, region_d)

compare_cl = recordlinkage.Compare()
compare_cl.string('region_name_echis', 'region_name_dhis2', method='jarowinkler', label='match_score')

regions_linked = compare_cl.compute(region_candidate_links, region_e, region_d)
regions_linked = regions_linked.reset_index()

regions_linked = regions_linked.merge(region_e.reset_index().rename({'index':'level_0'},axis=1), how='left', on='level_0')

regions_linked = regions_linked.merge(region_d.reset_index().rename({'index':'level_1'},axis=1), how='left', on='level_1')

regions_linked['ranked'] = regions_linked.groupby('level_0')['match_score'].rank(ascending=False)

regions_linked = regions_linked[regions_linked['ranked'] == 1]

# Add the mapped regions to the eCHIS dataframe
df_new_eCHIS_combined_with_mappings = df_new_eCHIS_combined.merge(regions_linked[['region_name_echis','region_name_dhis2']], how='left', left_on='region_name_echis', right_on='region_name_echis')

df_new_eCHIS_combined_with_mappings

Using recordlinkage to link zones

In [ ]:
zone_e = df_eCHIS_combined_with_mappings[['region_name_mfr', 'zone_name_echis']].drop_duplicates().dropna().reset_index(drop=True)
zone_m = df_mfr_updated[['region_name_mfr', 'zone_name_mfr']].drop_duplicates().dropna().reset_index(drop=True)

indexer = recordlinkage.Index()
indexer.block('region_name_mfr')
zone_candidate_links = indexer.index(zone_e, zone_m)

compare_cl = recordlinkage.Compare()
compare_cl.string('zone_name_echis', 'zone_name_mfr', method='jarowinkler', label='match_score') 

zones_linked = compare_cl.compute(zone_candidate_links, zone_e, zone_m) 
zones_linked = zones_linked.reset_index()
zones_linked = zones_linked.merge(zone_e.reset_index().rename({'index':'level_0'},axis=1), how='left', on='level_0')
zones_linked = zones_linked.merge(zone_m[['zone_name_mfr']].reset_index().rename({'index':'level_1'},axis=1), how='left', on='level_1')
zones_linked['ranked'] = zones_linked.groupby('level_0')['match_score'].rank(ascending=False, method='first')
zones_linked['confirmation_required'] = zones_linked[['ranked', 'match_score']].apply(lambda x : 1 if ((x.match_score >= 0.8) and (x.ranked == 1)) else None, axis=1)

# For zones where it is ranked as 1, but score is below 0.8, create a list of those (which will be manually checked)
zones_manual = zones_linked[(zones_linked['ranked'] == 1) & (zones_linked['confirmation_required'].isnull())]['level_0'].unique()

# Check if a zone from mfr is mapped to more than one from echis
zones_duplicate_matches = zones_linked[zones_linked['confirmation_required'] == 1].groupby('level_1').agg({'level_0':'nunique'}).reset_index()
zones_duplicate_matches = zones_duplicate_matches[zones_duplicate_matches['level_0'] > 1]['level_1']

zones_linked['duplicate_mfr_match'] = zones_linked[['confirmation_required','level_1']].apply(lambda x : 1 if ((x.confirmation_required == 1) and (x.level_1 in list(zones_duplicate_matches))) else None, axis=1)
zones_linked['manual_intervention'] = zones_linked['level_0'].apply(lambda x : 0 if x in zones_manual else None)

try:
    zones_linked = pd.read_excel(zone_mappings_file_path)
except:
    zones_linked.to_excel(zone_mappings_file_path, index=False)

# Manual intervention will be required at this stage to ensure zones are mapped correctly

print('Unique Zones: ', len(zones_linked[['level_0']].drop_duplicates()))
zones_linked = zones_linked[(zones_linked['confirmation_required'] == 1) | (zones_linked['manual_intervention'] == 1)]
print('Zones mapped: ', len(zones_linked))

df_eCHIS_combined_with_mappings = df_eCHIS_combined_with_mappings.merge(zones_linked[['region_name_mfr','zone_name_echis','zone_name_mfr']], how='left', left_on=['zone_name_echis','region_name_mfr'], right_on=['zone_name_echis','region_name_mfr'])
zones_linked

Using recordlinkage to link woredas

In [ ]:
woreda_e = df_eCHIS_combined_with_mappings[['region_name_mfr', 'zone_name_mfr', 'woreda_name_echis']].drop_duplicates().dropna().reset_index(drop=True)
woreda_m = df_mfr_updated[['region_name_mfr', 'zone_name_mfr', 'woreda_name_mfr']].drop_duplicates().dropna().reset_index(drop=True)

indexer = recordlinkage.Index()
indexer.block(['region_name_mfr', 'zone_name_mfr'], ['region_name_mfr', 'zone_name_mfr'])
woreda_candidate_links = indexer.index(woreda_e, woreda_m)

compare_cl = recordlinkage.Compare()
compare_cl.string('woreda_name_echis', 'woreda_name_mfr', method='jarowinkler', label='match_score')

woredas_linked = compare_cl.compute(woreda_candidate_links, woreda_e, woreda_m)
woredas_linked = woredas_linked.reset_index()
woredas_linked = woredas_linked.merge(woreda_e.reset_index().rename({'index':'level_0'},axis=1), how='left', on='level_0')
woredas_linked = woredas_linked.merge(woreda_m[['woreda_name_mfr']].reset_index().rename({'index':'level_1'},axis=1), how='left', on='level_1')
woredas_linked['ranked'] = woredas_linked.groupby('level_0')['match_score'].rank(ascending=False, method='first')
woredas_linked['confirmation_required'] = woredas_linked[['ranked', 'match_score']].apply(lambda x : 1 if ((x.match_score >= 0.8) and (x.ranked == 1)) else None, axis=1)

# For woredas where it is ranked as 1, but score is below 0.8, create a list of those (which will be manually checked)
woredas_manual = woredas_linked[(woredas_linked['ranked'] == 1) & (woredas_linked['confirmation_required'].isnull())]['level_0'].unique()

# Check if a woreda from mfr is mapped to more than one from echis
woreda_duplicate_matches = woredas_linked[woredas_linked['confirmation_required'] == 1].groupby('level_1').agg({'level_0':'nunique'}).reset_index()
woreda_duplicate_matches = woreda_duplicate_matches[woreda_duplicate_matches['level_0'] > 1]['level_1']

woredas_linked['duplicate_mfr_match'] = woredas_linked[['confirmation_required','level_1']].apply(lambda x : 1 if ((x.confirmation_required == 1) and (x.level_1 in list(woreda_duplicate_matches))) else None, axis=1)
woredas_linked['manual_intervention'] = woredas_linked['level_0'].apply(lambda x : 0 if x in woredas_manual else None)

try:
    woredas_linked = pd.read_excel(woreda_mappings_file_path)
except:
    woredas_linked.to_excel(woreda_mappings_file_path, index=False)

# Manual intervention will be required at this stage to ensure woredas are mapped correctly

print('Unique Woredas: ', len(woredas_linked[['level_0']].drop_duplicates()))
woredas_linked = woredas_linked[(woredas_linked['confirmation_required'] == 1) | (woredas_linked['manual_intervention'] == 1)]
print('Woredas mapped: ', len(woredas_linked))

df_eCHIS_combined_with_mappings = df_eCHIS_combined_with_mappings.merge(woredas_linked[['region_name_mfr','zone_name_mfr','woreda_name_echis','woreda_name_mfr']], how='left', left_on=['woreda_name_echis', 'zone_name_mfr','region_name_mfr'], right_on=['woreda_name_echis', 'zone_name_mfr','region_name_mfr'])
woredas_linked 